In [ ]:
class ConvMaker:
    
    def __init__(self,tweeterlist):
        self.list = tweeterlist

    def getKlmToUser(self):
        #makes a list of all klm tweets @ a person
        repListKlm = []
        #for loop which looks for any '@' in the text of the tweet
        for item in self.list.iterrows():
            if "@" in item[1]['text']:
                repListKlm.append(item[1])
        repKlm = pandas.DataFrame(repListKlm)
        ########################
        
        
        ########################
        #this makes a new time object which is a numerical value which gets higher the later the data
        value = 0
        current_date = 'null'
        userRepliedTo = []
        #gets the date and sees if it is a later date if it is it assigns it a higher value
        for tweet in repKlm.iterrows():
            if tweet[1]['created_at'][0:10] != current_date:
                current_date = tweet[1]['created_at'][0:10]
                value +=1
            tweet[1]['date'] = value
            #makes a new dataframe with the new date value included
            userRepliedTo.append(tweet[1])
        ########################
        
        
        ########################
        #turns it into a dataframe, extracts only the portion between the @ and first space and deletes @ nothing and @KLM...
        userRepliedTo = pandas.DataFrame(userRepliedTo)
        userRepliedTo['rep_name']= userRepliedTo['text'].str.extract(r'.*(@[a-zA-Z0-9_]*).*')
        userRepliedTo = userRepliedTo[userRepliedTo['rep_name']!='@']
        userRepliedTo = userRepliedTo[userRepliedTo['rep_name'].str.extract(r'@(KLM).*')!='KLM']
        userRepliedTo = userRepliedTo[userRepliedTo['rep_name'] !='@klm']
        ###########################
        
        
        ###########################
        #gets the names of users and puts them into a list
        userlist = list(userRepliedTo['rep_name'])
        #removes duplicates from the list
        userlist = list(dict.fromkeys(userlist))
        
        #turns the list into a dict with empty lists as values
        userdict ={}
        for user in userlist:
            userdict[user] = []

        #appends the tweets to their corresponding names, so now the dict contains all tweets of a person grouped toghether
        for tweet in userRepliedTo.iterrows():
            try:
                userdict[tweet[1]['rep_name']].append(tweet[1])
            except:
                print('non dict value appended')

        #turns the dict into a list of lists of dicts, sorted by unique person
        dfpersons = []
        for person in userdict.keys():
            dfpersons.append(userdict[person])
        ####################################  
        
        ####################################
        replist = []
        for person in dfpersons:
            person = pandas.DataFrame(person)
            for tweet in person.iterrows():
                bar = tweet[1]['in_reply_to_status_id_str']
                if bar != None:
                    replist.append(bar)

        return dfpersons,replist


In [ ]:
################################
#EXAMPLE
###############################
sentlist = RawDataUserFilter('json/filenames.txt','month1','KLMnumb shortlist.txt', 'json/').getKlmAllBB()
sentlist = FrameConcat(sentbyList).concatinator()
listOfTweetsSortByPerson = []
for i in range(0,len(sentlist)):
    listOfTweetsSortByPerson.append(ConvMaker(sentlist[i]).getKlmToUser())
pandas.DataFrame(a[0][0][15])

In [ ]:
class OriginTweet:
    
    def __init__(self,textlist,path,listOfReplies):
        self.text = textlist
        self.path = path
        self.rep = listOfReplies

    def getInitTweet(self):
        data = []
        for filename in self.text:
            linecount = 0
            with open(self.path+filename) as f:
                for line in f:
                    linecount +=1
                    try:
                        curline = json.loads(line)
                        if curline['id_str'] in self.rep:
                            data.append(curline)
                    except:
                        print("fault in line:"+str(linecount))
        data = pandas.DataFrame(data)
        return data


In [1]:
##########################
#EXAMPLE
##########################
textlist = RawDataUserFilter('json/filenames.txt','month1',3,'json/').textmaker()
path = 'json/'
listOfReplies = listOfTweetsSortByPerson[0][1]
OriginTweet(textlist,path,listOfReplies).getInitTweet()